In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [14]:
import json

files = ['news-windows.json', 'wikipedia-1word.json', 'wikipedia-windows.json']
left, right = [], []
for f in files:
    with open(f) as fopen:
        data = json.load(fopen)
        for d in data:
            if len(d) == 2:
                if len(d[0]) and len(d[1]):
                    left.append(d[0])
                    right.append(d[1])

In [15]:
PAD = 0
EOS = 1
UNK = 2
GO = 3
[PAD, EOS, UNK, GO]

[0, 1, 2, 3]

In [18]:
left_dict =  [PAD, EOS, UNK, GO] + sorted(set(list(''.join(left))) | set(list(''.join(right))))
left_dict = {c: no for no, c in enumerate(left_dict)}
rev_left_dict = {v: k for k, v in left_dict.items()}

In [25]:
left_dict

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 ' ': 4,
 '!': 5,
 '"': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 '+': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '0': 14,
 '1': 15,
 '2': 16,
 '3': 17,
 '4': 18,
 '5': 19,
 '6': 20,
 '7': 21,
 '8': 22,
 '9': 23,
 ':': 24,
 ';': 25,
 '?': 26,
 'A': 27,
 'B': 28,
 'C': 29,
 'D': 30,
 'E': 31,
 'F': 32,
 'G': 33,
 'H': 34,
 'I': 35,
 'J': 36,
 'K': 37,
 'L': 38,
 'M': 39,
 'N': 40,
 'O': 41,
 'P': 42,
 'Q': 43,
 'R': 44,
 'S': 45,
 'T': 46,
 'U': 47,
 'V': 48,
 'W': 49,
 'X': 50,
 'Y': 51,
 'Z': 52,
 'a': 53,
 'b': 54,
 'c': 55,
 'd': 56,
 'e': 57,
 'f': 58,
 'g': 59,
 'h': 60,
 'i': 61,
 'j': 62,
 'k': 63,
 'l': 64,
 'm': 65,
 'n': 66,
 'o': 67,
 'p': 68,
 'q': 69,
 'r': 70,
 's': 71,
 't': 72,
 'u': 73,
 'v': 74,
 'w': 75,
 'x': 76,
 'y': 77,
 'z': 78,
 '،': 79,
 '؟': 80,
 'ء': 81,
 'آ': 82,
 'أ': 83,
 'ؤ': 84,
 'إ': 85,
 'ئ': 86,
 'ا': 87,
 'ب': 88,
 'ة': 89,
 'ت': 90,
 'ث': 91,
 'ج': 92,
 'ح': 93,
 'خ': 94,
 'د': 95,
 'ذ': 96,
 'ر': 97,
 'ز': 98,
 'س': 99,
 'ش': 100,
 'ص': 1

In [19]:
class Encoder:
    def __init__(self, dict):
        self.dict = dict
        self.vocab_size = len(self.dict)

    def encode(self, s):
        s = [left_dict[c] for c in s] + [1]
        return s
    
    def decode(self, ids):
        return ''.join([rev_left_dict[i] for i in ids if i > 3])


encoder = Encoder(left_dict)

In [22]:
%%time

encoder.encode('saya')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.05 µs


[71, 53, 77, 53, 1]

In [23]:
encoder.decode([71, 53, 77, 53, 1])

'saya'

In [11]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry
from tqdm import tqdm

2022-05-31 15:00:58.005135: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [24]:
len(left_dict)

137

In [26]:
with open('jawi-set.json') as fopen:
    jawi = json.load(fopen)
jawi['train'][:10]

['كاوسن كڤولاوان سڤراتلي يڠ',
 'ڤرليمين ڤرسكوتوان اونتوق',
 'ڤنوبوهن تامن سينر',
 'ڤريڠكت كمنترين، كاتڽ.',
 'تله مندرم سباڽق تيڬ',
 'هاري اين،',
 'برتوليرنسي"',
 'مڠيسهكان',
 'سوڠ-قواڠ',
 'سيبير دڠن باجو بياسا.']

In [27]:
with open('rumi-set.json') as fopen:
    rumi = json.load(fopen)

In [29]:
train_X = rumi['train']
train_Y = jawi['train']

test_X = rumi['test']
test_Y = jawi['test']

In [30]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry
from tqdm import tqdm

2022-06-01 13:20:10.686239: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [33]:
@registry.register_problem
class Jawi(text_problems.Text2TextProblem):
    @property
    def approx_vocab_size(self):
        return len(left_dict)

    @property
    def is_generate_per_split(self):
        # generate_data will shard the data into TRAIN and EVAL for us.
        return False

    @property
    def dataset_splits(self):
        return [
            {'split': problem.DatasetSplit.TRAIN, 'shards': 20},
        ]

    def generate_samples(self, data_dir, tmp_dir, dataset_split):

        for i in tqdm(range(len(train_X))):
            l = encoder.encode(train_X[i])
            r = encoder.encode(train_Y[i])
            yield {'inputs': l, 'targets': r}

    def generate_encoded_samples(self, data_dir, tmp_dir, dataset_split):

        generator = self.generate_samples(data_dir, tmp_dir, dataset_split)
        for sample in generator:
            yield sample

In [34]:
import os
import tensorflow as tf

In [35]:
os.system('rm -rf t2t-rumi-jawi/data')
DATA_DIR = os.path.expanduser('t2t-rumi-jawi/data')
TMP_DIR = os.path.expanduser('t2t-rumi-jawi/tmp')

In [36]:
tf.gfile.MakeDirs(DATA_DIR)
tf.gfile.MakeDirs(TMP_DIR)

from tensor2tensor.utils import registry
from tensor2tensor import problems

PROBLEM = 'jawi'
t2t_problem = problems.problem(PROBLEM)
t2t_problem.generate_data(DATA_DIR, TMP_DIR)

/home/ubuntu/tf-nvidia/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/498106 [00:00<?, ?it/s]

INFO:tensorflow:Generating case 0.


INFO:tensorflow:Generating case 0.
 20%|█▉        | 98205/498106 [00:02<00:08, 44725.60it/s]

INFO:tensorflow:Generating case 100000.


INFO:tensorflow:Generating case 100000.
 40%|███▉      | 196924/498106 [00:04<00:06, 45045.99it/s]

...


INFO:tensorflow:Generated 498106 Examples


INFO:tensorflow:Shuffling data...


INFO:tensorflow:Shuffling data...


INFO:tensorflow:Data shuffled.


INFO:tensorflow:Data shuffled.


In [17]:
# tf.train.list_variables('t2t-phoneme/train-small/model.ckpt-1100')